<a href="https://colab.research.google.com/github/PhamMinhTuanGit/ScoringMultipleChoicesTestApp/blob/main/mamba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mamba Inference Script
by [Trelis.com](https://Trelis.com).

Find us on [YouTube](https://Youtube.com/@TrelisResarch).

Built from: [State spaces, Mamba on GitHub](https://github.com/state-spaces/mamba/blob/main/benchmarks/benchmark_generation_mamba_simple.py)

## Getting started
Select a GPU from Runtime -> Change Runtime type.

In [ ]:
!pip install causal-conv1d>=1.1.0 -q -U

In [ ]:
!pip install mamba-ssm -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!git clone https://github.com/state-spaces/mamba.git

Cloning into 'mamba'...
remote: Enumerating objects: 715, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 715 (delta 23), reused 15 (delta 15), pack-reused 683 (from 2)
Receiving objects: 100% (715/715), 1.55 MiB | 11.85 MiB/s, done.
Resolving deltas: 100% (383/383), done.


In [ ]:
%cd mamba

/content/mamba


In [ ]:
!echo "/usr/lib64-nvidia/" | sudo tee -a /etc/ld.so.conf.d/libcuda.conf
!ldconfig

/usr/lib64-nvidia/
/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_loader.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_opencl.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm_debug.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_level_zero.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libumf.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local

## One line example

In [ ]:
!python benchmarks/benchmark_generation_mamba_simple.py --model-name "Trelis/mamba-2.8b-slimpj-bf16" --prompt "Here is a python program to add the first five fibonnacci numbers:\n" --topp 0.9 --temperature 0.7 --repetition-penalty 1.2

Loading model Trelis/mamba-2.8b-slimpj-bf16
config.json: 100% 184/184 [00:00<00:00, 1.50MB/s]
Traceback (most recent call last):
  File "/content/mamba/benchmarks/benchmark_generation_mamba_simple.py", line 40, in <module>
    tokenizer = AutoTokenizer.from_pretrained(args.model_name)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py", line 992, in from_pretrained
    return tokenizer_class_fast.from_pretrained(pretrained_model_name_or_path, *inputs, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py", line 2046, in from_pretrained
    raise EnvironmentError(
OSError: Can't load tokenizer for 'Trelis/mamba-2.8b-slimpj-bf16'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with

## Explicit Example

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel

In [ ]:
# Set parameters
model_name = "Trelis/mamba-2.8b-slimpj-bf16" # or "clibrain/mamba-2.8b-instruct-openhermes"
# model_name = "clibrain/mamba-2.8b-instruct-openhermes"
promptlen = 100
genlen = 100
temperature = 0.01
topk = 1
topp = 1.0
repetition_penalty = 1.0
batch = 1
repeats = 3
device = "cuda"
dtype = torch.float16 #or use torch.bfloat16 for an A100 GPU

# Load model
print(f"Loading model {model_name}")
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
model = MambaLMHeadModel.from_pretrained(
    model_name,
    device=device,
    dtype=dtype)

model.eval()
print(f"Number of parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

In [ ]:
print(model.config)

In [ ]:
print(model)

In [ ]:
# prompt = "Question: Write a short python program that adds together the first five fib numbers.\n\nAnswer: Here is the answer:\n"  # Replace with your prompt if needed
prompt = "What planets are in our solar system?\n\nAnswer: Here is the answer:\n"  # Replace with your prompt if needed

torch.random.manual_seed(0)
if prompt is None:
    input_ids = torch.randint(1, 1000, (batch, promptlen), dtype=torch.long, device=device)
    attn_mask = torch.ones_like(input_ids, dtype=torch.long, device=device)
else:
    tokens = tokenizer(prompt, return_tensors="pt")
    input_ids = tokens.input_ids.to(device=device)
    attn_mask = tokens.attention_mask.to(device=device)
max_length = input_ids.shape[1] + genlen

fn = lambda: model.generate(
    input_ids=input_ids,
    max_length=max_length,
    cg=True,
    return_dict_in_generate=True,
    output_scores=True,
    enable_timing=False,
    temperature=temperature,
    top_k=topk,
    top_p=topp,
    eos_token_id=tokenizer.eos_token_id,
    repetition_penalty=repetition_penalty,
)

In [ ]:
print(tokenizer.eos_token_id)
eos_token = tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id)
print(eos_token)

In [ ]:
import time

out = fn()
if prompt is not None:
    print(prompt)
    print("\n")  # Print new lines
    # Decode and print the generated text
    decoded_sequences = tokenizer.batch_decode(out.sequences.tolist())
    for sequence in decoded_sequences:
        generated_text = sequence[len(tokenizer.decode(input_ids[0], skip_special_tokens=True)):]
        print(generated_text)

torch.cuda.synchronize()
start = time.time()
for _ in range(repeats):
    fn()
torch.cuda.synchronize()
print(f"Prompt length: {len(input_ids[0])}, generation length: {len(out.sequences[0]) - len(input_ids[0])}")
print(f"{model_name} prompt processing + decoding time: {(time.time() - start) / repeats * 1000:.0f}ms")

In [ ]:
def fibonacci(n):
    a, b = 0, 1
    for i in range(n):
        print(a)
        a, b = b, a + b

print(fibonacci(5))

### Passkey Retrieval (you must first have loaded the model above in the explicit example)

In [ ]:
# Define the passkey and text file
passkey = "(the passkey is 'u89dsnakj8')"
text_file = '../berkshire23.txt'

# Define the length limit for the text
len_limit = int(16000 * 0.8)  # 16k characters is about 4k tokens of context.

# Calculate the position to insert the passkey
n = int(len_limit / 5 * 0.5)  # Placement of the passkey as a % of total text.

# Read the text from the file
with open(text_file, 'r') as file:
    text = file.read()

# Split the text into words
words = text.split()

# Insert the passkey after the nth word
words.insert(n, passkey)

# Join back into a string and truncate to 'len_limit' characters
modified_text = ' '.join(words)[:len_limit]

# Define the prompt for the model
prompt = f'Respond with the passkey contained within the below text.\n\n{modified_text}\n\nRespond with the passkey contained within the above text.\n\nThe passkey is:'

# Prepare the input for the model
tokens = tokenizer(prompt, return_tensors="pt")
input_ids = tokens.input_ids.to(device=device)
attn_mask = tokens.attention_mask.to(device=device)
max_length = input_ids.shape[1] + 100  # Assuming you want to generate 100 tokens

# Generate function for Mamba model
def generate_mamba(input_ids, max_length):
    return model.generate(
        input_ids=input_ids,
        max_length=max_length,
        cg=True,
        return_dict_in_generate=True,
        output_scores=True,
        enable_timing=False,
        temperature=0.01,
        top_k=1,
        top_p=1.0,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.0,
    )

# Run the generation
out = generate_mamba(input_ids, max_length)

# Decode and print the generated text
decoded_sequences = tokenizer.batch_decode(out.sequences.tolist())
for sequence in decoded_sequences:
    generated_text = sequence[len(tokenizer.decode(input_ids[0], skip_special_tokens=True)):]
    print(generated_text)

# Optionally, measure the time taken for generation
torch.cuda.synchronize()
start = time.time()
for _ in range(3):  # Number of repeats
    generate_mamba(input_ids, max_length)
torch.cuda.synchronize()
end = time.time()
print(f"{model_name} prompt processing + decoding time: {(end - start) / 3 * 1000:.0f}ms")

## OpenHermes Inference

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel

CHAT_TEMPLATE_ID = "HuggingFaceH4/zephyr-7b-beta"

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_name = "clibrain/mamba-2.8b-instruct-openhermes"

eos_token = "<|endoftext|>"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.eos_token = eos_token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = AutoTokenizer.from_pretrained(CHAT_TEMPLATE_ID).chat_template

# Uncomment the below if you haven't loaded a model above.
# model = MambaLMHeadModel.from_pretrained(
#         model_name, device=device, dtype=torch.float16)

messages = []
# prompt = "Tell me 5 sites to visit in Spain"
# prompt = "What are the planets in our solar system"
prompt = "Write a piece of python code to add the first five fibonacci numbers"
messages.append(dict(role="user", content=prompt))

input_ids = tokenizer.apply_chat_template(
            messages, return_tensors="pt", add_generation_prompt=True
).to(device)

out = model.generate(
    input_ids=input_ids,
    max_length=250,
    temperature=0.9,
    top_p=0.7,
    eos_token_id=tokenizer.eos_token_id,
)

decoded = tokenizer.batch_decode(out)
assistant_message = (
    decoded[0].split("<|assistant|>\n")[-1].replace(eos_token, "")
)

print(assistant_message)

In [ ]:
fibonacci_sequence = [0, 1]
for i in range(2, 6):
    fibonacci_sequence.append(fibonacci_sequence[i-1] + fibonacci_sequence[i-2])
print(fibonacci_sequence)

## Uploading Models to Hub

In [ ]:
new_repo_name="Trelis/mamba-2.8b-slimpj"

model.save_pretrained(new_repo_name)

In [ ]:
!pip install huggingface-hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import HfApi, upload_folder
import os

# Set the repository name and local folder path
repo_id = "Trelis/mamba-2.8b-slimpj"
folder_path = "./Trelis/mamba-2.8b-slimpj"  # Adjust this path to your local folder

# Optionally, set your Hugging Face authentication token
# You can retrieve your token from your Hugging Face account (https://huggingface.co/settings/tokens)
# If you don't set the token here, the function will use the token stored on your machine
# token = None  # Replace with your token if needed

# Upload the folder to the repository
upload_url = upload_folder(
    repo_id=repo_id,
    folder_path=folder_path,
    # token=token  # Include this if you have set a token above
)

print(f"Folder uploaded successfully. You can view it at: {upload_url}")

In [ ]:
%cd ../../
%ls -l -a